<a href="https://colab.research.google.com/github/onurburakozdemir/FIZ437E/blob/main/HW3_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import torch
from torch.autograd import Variable
import numpy as np

In [15]:
train_dataset2 = datasets.MNIST(root='./data', train=True, download=True)
test_dataset2 = datasets.MNIST(root='./data', train=False)

In [16]:
MNIST_train_dataset = []
MNIST_train_label = []
for i, (img, label) in enumerate(train_dataset2):
  img_arry = np.array(img)
  dtype = img_arry.dtype
  h, w = img_arry.shape
  img_arry = np.reshape(img_arry, [h, w, 1])
  img_arry = np.concatenate([img_arry, img_arry, img_arry], axis=2)
  MNIST_train_dataset.append(img_arry)
  MNIST_train_label.append(label)

MNIST_test_dataset = []
MNIST_test_label = []
for i, (img, label) in enumerate(test_dataset2):
  img_arry = np.array(img)
  dtype = img_arry.dtype
  h, w = img_arry.shape
  img_arry = np.reshape(img_arry, [h, w, 1])
  img_arry = np.concatenate([img_arry, img_arry, img_arry], axis=2)
  MNIST_test_dataset.append(img_arry)
  MNIST_test_label.append(label)

In [17]:
cMNIST_train_dataset = []
cMNIST_train_label = []
for i, (img, label) in enumerate(train_dataset2):
  img_arry = np.array(img)
  dtype = img_arry.dtype
  h, w = img_arry.shape
  img_arry = np.reshape(img_arry, [h, w, 1])
  if i < 15000:
    img_arry = np.concatenate([img_arry, 
                               np.zeros((h, w, 2), dtype=dtype)], axis=2)
  elif i < 30000:
    img_arry = np.concatenate([np.zeros((h, w, 2), dtype=dtype),
                               img_arry], axis=2)
  elif i < 45000:
    img_arry = np.concatenate([np.zeros((h, w, 1), dtype=dtype), img_arry, 
                               np.zeros((h, w, 1), dtype=dtype)], axis=2)
  else:
    img_arry = np.concatenate([img_arry, img_arry, 
                          np.zeros((h, w, 1), dtype=dtype)], axis=2)
  cMNIST_train_dataset.append(img_arry)
  cMNIST_train_label.append(label)

cMNIST_test_dataset = []
cMNIST_test_label = []
for i, (img, label) in enumerate(test_dataset2):
  img_arry = np.array(img)
  dtype = img_arry.dtype
  h, w = img_arry.shape
  img_arry = np.reshape(img_arry, [h, w, 1])
  if i < 2500:
    img_arry = np.concatenate([img_arry,
                               np.zeros((h, w, 2), dtype=dtype)], axis=2)
  elif i < 5000:
    img_arry = np.concatenate([np.zeros((h, w, 2), dtype=dtype),
                               img_arry], axis=2)
  elif i < 7500:
    img_arry = np.concatenate([np.zeros((h, w, 1), dtype=dtype), img_arry, 
                               np.zeros((h, w, 1), dtype=dtype)], axis=2)
  else:
    img_arry = np.concatenate([img_arry, img_arry, 
                          np.zeros((h, w, 1), dtype=dtype)], axis=2)
  cMNIST_test_dataset.append(img_arry)
  cMNIST_test_label.append(label)

In [18]:
c_train = [(img.astype(np.float32), 
             cMNIST_train_label[i]) 
             for i, img in enumerate(cMNIST_train_dataset)]

c_test = [(img.astype(np.float32), 
             cMNIST_test_label[i]) 
             for i, img in enumerate(cMNIST_test_dataset)]

bw_train = [(img.astype(np.float32), 
             MNIST_train_label[i]) 
             for i, img in enumerate(MNIST_train_dataset)]

bw_test = [(img.astype(np.float32), 
             MNIST_test_label[i]) 
             for i, img in enumerate(MNIST_test_dataset)]

In [19]:
c_train_loader = DataLoader(c_train, batch_size=32)
c_test_loader = DataLoader(c_test, batch_size=10000)
bw_train_loader = DataLoader(bw_train, batch_size=32)
bw_test_loader = DataLoader(bw_test, batch_size=10000)

In [20]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28*28*3, 64)
        self.l2 = nn.Linear(64,64)
        self.l3 = nn.Linear(64,10)
        # dropout prevents overfitting of data
        self.do = nn.Dropout(0.1)

    def forward(self,x):
        x = x.view(-1, 28 * 28 * 3)
        # hidden layer, with relu activation function
        h1 = nn.functional.relu(self.l1(x))
        # hidden layer, with relu activation function
        h2 =  nn.functional.relu(self.l2(h1))
        # dropout layer
        do = self.do(h2+h1)
        # output layer
        logits = self.l3(do)
        return logits # initialize the NN

In [21]:
model = ResNet()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

Normal MNIST datasetinde eğitildi ve test edildi

In [22]:
train_accuracies = []
train_loss = []
test_accuracies = []
test_loss = []
n_epoch = 10
for i in range(n_epoch):
  accuracies=[]
  losses=[]
  print(i, end=' ')
  for j, (images, labels) in enumerate(bw_train_loader):
    optimizer.zero_grad()
    outputs = model(images/255)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
  train_accuracies.append(torch.tensor(accuracies).mean())
  train_loss.append(torch.tensor(losses).mean())
  print(torch.tensor(accuracies).mean(), end=' ')
  print(torch.tensor(losses).mean(), end=' ')

  with torch.no_grad():
    for j, (images, labels) in enumerate(bw_test_loader):
      outputs = model(images/255)
      loss = criterion(outputs, labels)

    train_accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
    train_loss.append(loss.item())
    print(labels.eq(outputs.detach().argmax(dim=1)).float().mean(), end=' ')
    print(loss.item())


0 tensor(0.8425) tensor(0.6004) tensor(0.9029) 0.3337898552417755
1 tensor(0.9144) tensor(0.3013) tensor(0.9254) 0.2559989094734192
2 tensor(0.9305) tensor(0.2427) tensor(0.9347) 0.22160862386226654
3 tensor(0.9410) tensor(0.2055) tensor(0.9426) 0.19433791935443878
4 tensor(0.9480) tensor(0.1806) tensor(0.9486) 0.16881237924098969
5 tensor(0.9536) tensor(0.1599) tensor(0.9531) 0.15341408550739288
6 tensor(0.9591) tensor(0.1436) tensor(0.9573) 0.14373748004436493
7 tensor(0.9622) tensor(0.1315) tensor(0.9594) 0.13502110540866852
8 tensor(0.9651) tensor(0.1207) tensor(0.9623) 0.1256025731563568
9 tensor(0.9669) tensor(0.1133) tensor(0.9622) 0.12460730969905853


In [23]:
# model = Logistic_Reg_Model(28*28*3)
model2 = ResNet()
# model = LogisticRegression(28*28*3, 10)
# model = MyNet(28*28*3, 100, 50, 10)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

CMNIST'de eğitildi ve test edildi

In [26]:
train_accuracies = []
train_loss = []
test_accuracies = []
test_loss = []
n_epoch = 10
for i in range(n_epoch):
  accuracies=[]
  losses=[]
  print(i, end=' ')
  for j, (images, labels) in enumerate(c_train_loader):
    optimizer.zero_grad()
    outputs = model2(images/255)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
  train_accuracies.append(torch.tensor(accuracies).mean())
  train_loss.append(torch.tensor(losses).mean())
  print(torch.tensor(accuracies).mean(), end=' ')
  print(torch.tensor(losses).mean(), end=' ')

  with torch.no_grad():
    for j, (images, labels) in enumerate(c_test_loader):
      outputs = model2(images/255)
      loss = criterion(outputs, labels)

    train_accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
    train_loss.append(loss.item())
    print(labels.eq(outputs.detach().argmax(dim=1)).float().mean(), end=' ')
    print(loss.item())

0 tensor(0.0831) tensor(2.3127) tensor(0.0759) 2.315528154373169
1 tensor(0.0829) tensor(2.3126) tensor(0.0781) 2.315495014190674
2 tensor(0.0835) tensor(2.3127) tensor(0.0774) 2.315092086791992
3 tensor(0.0834) tensor(2.3126) tensor(0.0803) 2.314739942550659
4 tensor(0.0829) tensor(2.3127) tensor(0.0781) 2.3150243759155273
5 tensor(0.0826) tensor(2.3126) tensor(0.0772) 2.3143301010131836
6 tensor(0.0838) tensor(2.3128) tensor(0.0773) 2.3150463104248047
7 tensor(0.0840) tensor(2.3126) tensor(0.0773) 2.3153791427612305
8 tensor(0.0829) tensor(2.3127) tensor(0.0783) 2.3151803016662598
9 tensor(0.0824) tensor(2.3127) tensor(0.0729) 2.3152458667755127


In [27]:
# model = Logistic_Reg_Model(28*28*3)
model3 = ResNet()
# model = LogisticRegression(28*28*3, 10)
# model = MyNet(28*28*3, 100, 50, 10)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

CMNIST'de eğitildi ve Normal MNIST'de test edildi

In [28]:
train_accuracies = []
train_loss = []
test_accuracies = []
test_loss = []
n_epoch = 10
for i in range(n_epoch):
  accuracies=[]
  losses=[]
  print(i, end=' ')
  for j, (images, labels) in enumerate(c_train_loader):
    optimizer.zero_grad()
    outputs = model3(images/255)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
  train_accuracies.append(torch.tensor(accuracies).mean())
  train_loss.append(torch.tensor(losses).mean())
  print(torch.tensor(accuracies).mean(), end=' ')
  print(torch.tensor(losses).mean(), end=' ')

  with torch.no_grad():
    for j, (images, labels) in enumerate(bw_test_loader):
      outputs = model3(images/255)
      loss = criterion(outputs, labels)

    train_accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
    train_loss.append(loss.item())
    print(labels.eq(outputs.detach().argmax(dim=1)).float().mean(), end=' ')
    print(loss.item())

0 tensor(0.0926) tensor(2.3046) tensor(0.0905) 2.2959554195404053
1 tensor(0.0925) tensor(2.3049) tensor(0.0916) 2.2955777645111084
2 tensor(0.0935) tensor(2.3047) tensor(0.0902) 2.2963926792144775
3 tensor(0.0934) tensor(2.3048) tensor(0.0914) 2.2966854572296143
4 tensor(0.0923) tensor(2.3046) tensor(0.0885) 2.2959213256835938
5 tensor(0.0934) tensor(2.3046) tensor(0.0913) 2.296086311340332
6 tensor(0.0926) tensor(2.3046) tensor(0.0906) 2.2961907386779785
7 tensor(0.0928) tensor(2.3047) tensor(0.0931) 2.2961854934692383
8 tensor(0.0940) tensor(2.3046) tensor(0.0926) 2.296344041824341
9 tensor(0.0931) tensor(2.3047) tensor(0.0930) 2.2959961891174316


In [29]:
# model = Logistic_Reg_Model(28*28*3)
model4 = ResNet()
# model = LogisticRegression(28*28*3, 10)
# model = MyNet(28*28*3, 100, 50, 10)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [30]:
train_accuracies = []
train_loss = []
test_accuracies = []
test_loss = []
n_epoch = 10
for i in range(n_epoch):
  accuracies=[]
  losses=[]
  print(i, end=' ')
  for j, (images, labels) in enumerate(bw_train_loader):
    optimizer.zero_grad()
    outputs = model4(images/255)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
  train_accuracies.append(torch.tensor(accuracies).mean())
  train_loss.append(torch.tensor(losses).mean())
  print(torch.tensor(accuracies).mean(), end=' ')
  print(torch.tensor(losses).mean(), end=' ')

  with torch.no_grad():
    for j, (images, labels) in enumerate(c_test_loader):
      outputs = model4(images/255)
      loss = criterion(outputs, labels)

    train_accuracies.append(labels.eq(outputs.detach().argmax(dim=1)).float().mean())
    train_loss.append(loss.item())
    print(labels.eq(outputs.detach().argmax(dim=1)).float().mean(), end=' ')
    print(loss.item())

0 tensor(0.1045) tensor(2.3142) tensor(0.1102) 2.3083343505859375
1 tensor(0.1045) tensor(2.3141) tensor(0.1134) 2.308250665664673
2 tensor(0.1050) tensor(2.3144) tensor(0.1099) 2.3090243339538574
3 tensor(0.1043) tensor(2.3143) tensor(0.1112) 2.308685541152954
4 tensor(0.1045) tensor(2.3143) tensor(0.1090) 2.3083908557891846
5 tensor(0.1045) tensor(2.3141) tensor(0.1094) 2.308593988418579
6 tensor(0.1049) tensor(2.3144) tensor(0.1132) 2.308384656906128
7 tensor(0.1037) tensor(2.3144) tensor(0.1089) 2.3083813190460205
8 tensor(0.1045) tensor(2.3142) tensor(0.1109) 2.30826473236084
9 tensor(0.1047) tensor(2.3143) tensor(0.1115) 2.308321237564087
